<a href="https://colab.research.google.com/github/herculeslyndel/python-for-devops/blob/master/Python_for_DevOps_Chapter_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

```
Chapter 3: Working with the command line

GUI tools are good but command line is for DevOps
```

```
working with shell

    sys
    os
    subprocess
```

```
talking to interpreter with sys module

variables and methods 
associated with python interpreter

defines big endium little endium


sys.byteorder attribute shows byte order of current architecture

sys.getsizeof
sys.platform 
    which OS
sys.version_info
```

In [1]:
import sys
sys.byteorder

'little'

In [2]:
sys.getsizeof(list(range(10))), sys.getsizeof(list(range(1000)))

(200, 9112)

In [3]:
sys.getsizeof(range(10)), sys.getsizeof(range(1000))

(48, 48)

In [4]:
sys.platform

'linux'

In [5]:
sys.version_info

sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)

In [6]:
if sys.version_info.major < 3:
    print('u r running python 2...')
elif sys.version_info.minor < 7:
    print('u r running python 3 but not 3.7++')


u r running python 3 but not 3.7++


```
dealing with the OS using he os module

    most common usage is get settings from env vars for logging levels or api keys
```

In [7]:
import os
os.getcwd()

'/content'

In [8]:
os.chdir('/tmp')
os.getcwd()

'/tmp'

In [9]:
os.environ.get('LOGLEVEL')

In [10]:
os.environ['LOGLEVEL'] = 'DEBUG'

In [11]:
os.environ.get('LOGLEVEL')

'DEBUG'


```
Last login: Sat Jun 27 19:29:21 on ttys001
(base) stevedepp@Steves-MBP-2 ~ % python3
Python 3.7.6 (default, Jan  8 2020, 13:42:34) 
[Clang 4.0.1 (tags/RELEASE_401/final)] :: Anaconda, Inc. on darwin
Type "help", "copyright", "credits" or "license" for more information.
>>> import os
>>> os.getlogin()
'stevedepp'
>>> 
```

In [12]:
os.getlogin()

OSError: ignored

```
spawn process with subprocdess module

run application outside python from within python code
    built in shell commands
    bash scripts
    'spawn' a process and run commands within it

with subprocess, 
    you run favorite shell command or other command-line software
    you collect its output from within python

    most times use 'subprocess.run' to spawn a process
```

In [13]:
import subprocess

https://stackoverflow.com/questions/53209127/subprocess-unexpected-keyword-argument-capture-output/53209196


In [14]:
from subprocess import PIPE

cp = subprocess.run(["ls", "-l", "/dev/null"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
cp.stdout

'crw-rw-rw- 1 root root 1, 3 Jun 29 13:41 /dev/null\n'

```
this doesnt work until python 3.7 is installed
capture_output is same as setting capture stdout and capture stderr as above
```

In [15]:
cp = subprocess.run(['ls', '-l'],
                    capture_output=True,
                    universal_newlines=True)
cp.stdout

TypeError: ignored

In [16]:
cp = subprocess.run(["ls", "/doesnotexist", "/dev/null"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
cp.stderr

"ls: cannot access '/doesnotexist': No such file or directory\n"

```
better error handling with check parameter
this is how its meant to look ```

In [17]:
cp = subprocess.run(["ls", "/doesnotexist", "/dev/null"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True,  check=True)
cp.stderr

CalledProcessError: ignored

```
creating command line tools

    invoke Python script using Python
    1st step in creating command line tools is 
    separating code that should only run when invoked on the command line
```

In [18]:
os.chdir('/content')

```
here it runs on import
leaving out
    #!/usr/bin/env_python
```

In [19]:
import os
os.getcwd()

'/content'

In [20]:
%%writefile always_say_it.py
def say_it():
    greeting = 'Hello'
    target = 'Joe'
    message = f'{greeting} {target}'
    print(message)

say_it()

Writing always_say_it.py


In [21]:
!python always_say_it.py

Hello Joe


In [22]:
import always_say_it

Hello Joe


In [23]:
always_say_it.say_it()

Hello Joe


```
here it wont run on import because of __name__ test
leaving out
    #!/usr/bin/env python
```

In [24]:
%%writefile only_say_it_when_invoked.py
def say_it_when_invoked():
    greeting = 'Hello'
    target = 'Joseph'
    message = f'{greeting} {target}'
    print(message)
__name__
if __name__ == '__main__':
    say_it_when_invoked()
else:
    print('not invoked')
    print(__name__, 'inside test')
print(__name__,'outside test')

Writing only_say_it_when_invoked.py


```
not sure why here __name__ prints "__main__ outside test" 
but in the test __name__ == "__main__" is False

its because __name__ == __main__ tests if running from command line
```

In [25]:
!python only_say_it_when_invoked.py

Hello Joseph
__main__ outside test


In [26]:
import only_say_it_when_invoked

not invoked
only_say_it_when_invoked inside test
only_say_it_when_invoked outside test


In [27]:
only_say_it_when_invoked.say_it_when_invoked()

Hello Joseph


In [28]:
from only_say_it_when_invoked import say_it_when_invoked
say_it_when_invoked()

Hello Joseph


```
making your shell script executable
using this eliminate explicit call python when run script
    #!/usr/bin/env python
and you need to make it executable
    chmod +x say_it.py
```

In [29]:
%%writefile say_it_wo_python.py
#!/usr/bin/env python
def say_it_wo_python_function():
    greeting = 'Hello'
    target = 'Joseph!'
    message = f'{greeting} {target}'
    print(message)
__name__
if __name__ == '__main__':
    say_it_wo_python_function()
else:
    print('not invoked')
    print(__name__, 'inside test')
print(__name__,'outside test')

Writing say_it_wo_python.py


In [30]:
!chmod +x say_it_wo_python.py

In [31]:
!ls ./

always_say_it.py	     __pycache__  say_it_wo_python.py
only_say_it_when_invoked.py  sample_data


In [32]:
!./say_it_wo_python.py

Hello Joseph!
__main__ outside test


```
using sys.argv

    now that we've seaparteed code run only when invoked on command line...
    next step = accept command-line arguments
    if tool does only one thing then no need, 
        but if does more than one def 
        or if def takes args
        then need args for command line tools

    argv attribute of sys module
        simplest to process args from command line
        attribute = list of args pass to python script: 1st list item = module name

        ex 3.2 shows help msg and accpt args to the func 
            1 test if running from command line
            2 default values set here
            3 check for --help string in list of args
            4 exits after printing help msg
            5 position of the value after the flag which shoudl be the assoc value
            6 test the arg list is long enough; not so if flag provided without a value
            7 vall func w values as modif by args


```

In [33]:
%%writefile sys_argv_module.py
#!/usr/bin/env python
"""
Simple command-line tool using sys.argv
"""
import sys

if __name__ == '__main__':
    print(f"The first argument:  '{sys.argv[0]}'")
    print(f"The second argument: '{sys.argv[1]}'")
    print(f"The third argument: '{sys.argv[2]}'")
    print(f"The fourth argument: '{sys.argv[3]}'")

Writing sys_argv_module.py


In [34]:
!!chmod +x sys_argv_module.py
!./sys_argv_module.py --a-flag some-value 13

The first argument:  './sys_argv_module.py'
The second argument: '--a-flag'
The third argument: 'some-value'
The fourth argument: '13'


In [35]:
%%writefile sys_argv_module_2.py
#!/usr/bin/env python

"""
Simple command-line tool using sys.argv
"""
import sys
def say_it(greeting, target):
    message = f'{greeting} {target}'
    print(message)

if __name__ == '__main__': #1
    greeting = 'Hello' #2
    target = 'Joe'

if '--help' in sys.argv: #3
    help_message = f"usage: {sys.argv[0]} --name <NAME> --greeting <GREETING>"
    print(help_message)
    sys.exit() #4

if '--name' in sys.argv:
    # get position after flag
    name_index = sys.argv.index('--name') + 1 #5
    if name_index < len(sys.argv):
        name = sys.argv[name_index] #6

if '--greeting' in sys.argv:
    # get position after greeting flag
    greeting_index = sys.argv.index('--greeting') + 1
    if greeting_index < len(sys.argv):
        greeting = sys.argv[greeting_index]

say_it(greeting, name)

Writing sys_argv_module_2.py


In [36]:
!!chmod +x sys_argv_module_2.py
!./sys_argv_module_2.py --help

usage: ./sys_argv_module_2.py --name <NAME> --greeting <GREETING>


In [37]:
!!chmod +x sys_argv_module_2.py
!./sys_argv_module_2.py --name Sally --greeting Bonjour

Bonjour Sally


```
problems with ex 3.2 solution
    user misspells / miscapitalizes a flag, flag is ignored
```

```
using argparse

    part of python SL

    add_argument method
        1st arg is name of new arg = command flag
            if begins -- then treated as optional
            otherwise positional command

    parsed arguments opbject 
        with arguments as attributes that can use as input

    1 create parser obj w doc message
    2 add pos based command w help msg
    3 add optional arg
    4 store optional arg as boolean value
    5 use parser to parse args
    6 access arg values by name
        optional arg name as the -- removed

In [38]:
%%writefile simple_argparse.py
#!/usr/bin/env python
"""
command-line tool using argparse
"""
import argparse
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Echo your input') # 1
    parser.add_argument('message', # 2
                        help='message to echo')
    
    parser.add_argument('--twice', '-t', # 3
                        help='do it twice',
                        action='store_true') # 4
    
    args = parser.parse_args() # 5

    print(args.message) # 6
    if args.twice:
        print(args.message)

Writing simple_argparse.py


In [39]:
!!chmod +x simple_argparse.py
!./simple_argparse.py hello --twice

hello
hello


```
looks like --twice has no effect here
```

In [40]:
!./simple_argparse.py --help --twice

usage: simple_argparse.py [-h] [--twice] message

Echo your input

positional arguments:
  message      message to echo

optional arguments:
  -h, --help   show this help message and exit
  --twice, -t  do it twice


```
using argparse

    1 create top level parser
    2 add top level argument that is used with any command under this parsers hierarchy
    3 create subparser object to hold subparsers
        dest is name of attribute to select subparser
    4 add subparser for ships
    5 add command to ships subparser
        choices parameter gives list of choices for the command
    6 add a subparser for sailors
    7 add required positional arg to sailors subparser
    8 check which subparser is used by checking the func value

one top level optional arg = twice
two subparsers
    each has commands and flags
argparser creates help msg hierarchy displayed w the --help flag
    
```

In [41]:
%%writefile argparse_example.py
#!/usr/bin/env python

"""
Command-line tool using argparse
"""

import argparse

def sail():
    ship_name = 'Your ship'
    print(f"{ship_name} is setting sail")

def list_ships():
    ships = ['john b', 'yankee clipper', 'pequod']
    print(f"ships: {','.join(ships)}")

def greet(greeting, name):
    message = f'{greeting} {name}'
    print(message)

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='maritime control') #1

    parser.add_argument('--twice', '-t', #2 
                        help='do it twice',
                        action='store_true')
    subparsers = parser.add_subparsers(dest='func') #3
    ship_parser = subparsers.add_parser('ships', #4
                                     help='ship related commands')
    ship_parser.add_argument('command', #5
                             choices=['list', 'sail'])

    sailor_parser = subparsers.add_parser('sailors',
                                          help='talk to a sailor')
    sailor_parser.add_argument('name', #7
                               help='sailors name')
    sailor_parser.add_argument('--greeting', '-g',
                               help='greeting',
                               default='ahoy there')
    
    args = parser.parse_args()
    if args.func == 'sailors':
        greet(args.greeting, args.name)
    elif args.command == 'list':
        list_ships()
    else:
        sail()

Writing argparse_example.py


In [42]:
!!chmod +x argparse_example.py
!./argparse_example.py --help

usage: argparse_example.py [-h] [--twice] {ships,sailors} ...

maritime control

positional arguments:
  {ships,sailors}
    ships          ship related commands
    sailors        talk to a sailor

optional arguments:
  -h, --help       show this help message and exit
  --twice, -t      do it twice


In [43]:
!./argparse_example.py ships --help

usage: argparse_example.py ships [-h] {list,sail}

positional arguments:
  {list,sail}

optional arguments:
  -h, --help   show this help message and exit


```
function decorators

    function syntax
    takes other functions as arguments
        any function can take another function as an argument
        since all functions are objects
    decorator syntax gives a clean way to use functions as args

    so here decorator is called with the wrapped function as argument
    the decorator defines the wrapper for the wrapped function
        in that definitino the wrapper calls the wrapped function
        but this is only defining the wrapper
        then 
            the decorator calls the wraapper in its return statement 
            ****but it doesnt call it as wrapper()
            it calls it as wrapper which means it it not executed
            it returns the function compiled but not executed
                which calls to the wrapped funciton inside

    define the function
    and call it as an arg to the wrapper
```


In [44]:
def some_decorator(wrapped_function):
    def wrapper():
        print('do something before calling wrapped function')
        wrapped_function()
        print('do something after calling wrapped function')
    return wrapper

In [46]:
def foobat():
    print('foobat')

In [47]:
f = some_decorator(foobat)
f

<function __main__.some_decorator.<locals>.wrapper>

In [48]:
f()

do something before calling wrapped function
foobat
do something after calling wrapped function


```
decorator syntax simplifies this
by decorating the function with the @decorator_name

allows u to call wrapped function with its name rather than decorator name
there are prebuilt functions intended as decorators in PSL
    staticMethod
    classMethod

```

In [50]:
@some_decorator
def batfoo():
    print('batfoo')

In [51]:
batfoo()


do something before calling wrapped function
batfoo
do something after calling wrapped function


```
uaing click



```

In [52]:
%%writefile simple_click.py
#!/usr/bin/env python

"""
Simple Click ex
"""
import click
@click.command()
@click.option('--greeting', default='hiya', help='how do you want to greet?')
@click.option('--name', default='tammy', help='who do you want to greet?')
def greet(greeting, name):
    print(f"{greeting} {name}")

if __name__ == "__main__":
    greet()

Writing simple_click.py


In [56]:
!!chmod +x simple_click.py
!./simple_click.py --greeting Privet --name Peggy

Privet Peggy


In [57]:
!./simple_click.py --help 

Usage: simple_click.py [OPTIONS]

Options:
  --greeting TEXT  how do you want to greet?
  --name TEXT      who do you want to greet?
  --help           Show this message and exit.


```
more complex code with nesting of commands
    1 top level group under which other groups / commands can be def
    2 func for top level group = cli
        the click.group method transforms the cli func into a group
    3 group to hold ships command
    4 add ships group as command to top level cli group
        the cli func is now a group with add_command method 
    5 add a command to ships group
        the ships.command is used not click.command
    6 add a command to cli group
    7 call top level group 
```

In [64]:
%%writefile click_example.py
#!/usr/bin/env python

"""
Command-line tool using click
"""

import click

@click.group() #1
def cli(): #2
    pass

@click.group(help='ship related commands') #3
def ships():
    pass

cli.add_command(ships) #4

@ships.command(help='sail a ship') #5
def sail():
    ship_name = 'your ship'
    print(f"{ship_name} is setting sail")

@ships.command(help='list all ships')
def list_ships():
    ships = ['john b', 'yankee clipper', 'pequod']
    print("ships: {','.join(ships)}")

@cli.command(help='Talk to a sailor') #6 
@click.option('--greeting', default='ahoy there', help='greeting for sailor')
@click.argument('name')
def sailors(greeting, name):
    message = f'{greeting}, {name}'
    print(message)

if __name__ == '__main__':
    cli() #7

Overwriting click_example.py


In [65]:
!!chmod +x click_example.py
!./click_example.py --help

Usage: click_example.py [OPTIONS] COMMAND
                        [ARGS]...

Options:
  --help  Show this message and exit.

Commands:
  sailors  Talk to a sailor
  ships    ship related commands


```
defining classes
```

```
fire
```

```
implementing plugins
```

```
case study: turbocharging python with command line tools
```

```
using the numba just in time JIT compiler
```

```
using the GPU with CUDA pyton
```

```
running true multicore multithreaded python using numba
```


```
kmeans clustering
```